# 6. Classify companies and conduct statistical tests 
-------------------
Group 3 , September 24, 2022
1. Gezhi Cheng, 
2. Haowei Lee, 
3. Ziyi Liu, 
4. VS Chaitanya Madduri

> <i>Description: The program in this notebook
- classified records into different groups based on two dimension
    1. frequent restructuring
    2. positive change in profitability
- conduct the classification for three consecutive years
- conduct statistical tests
</i>


<div class="alert alert-block alert-info">
    <b>Tip:</b> # Please run this notebook in the colab . 
</div> 

### Pre requisites: 
1. And add the shortcut of the drive link :https://drive.google.com/drive/folders/1X4UdGsQiHVWSr63FRiz8rwOuWW5Ua8uI?usp=sharing to your personal drive.


Due to the huge files we have used our personal google drive folders to save the files.

Files:
- restructuring_data.csv - Data of company information and classification on frequent restructuring


### Output files:

Files:
None

## Load dependencies

In [1]:
# Mount the program to the folder on Google Drive to get access to other files on cloud
from google.colab import drive
drive.mount('/content/drive')

# import python packages
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# define the path of the root folder and of the data
DIR_PATH = "/content/drive/MyDrive/SPM_files/"  # take the current folder as root folder
RESTRUCTURING_DATA_PATH = "restructuring_data.csv"
LIMIT =3

COLUMNS_ORDER = ["Company_Key", "Year", 'compamy_name', "profitability", 'profitability_next_1',
       'profitability_next_2', 'profitability_next_3', 'change_next_1',
       'change_next_2', 'change_next_3', 'is_restructuring']

IS_RESTRUCTURING = "is_restructuring"
POSITIVE_CHANGE_1 = "positive_change_1"
POSITIVE_CHANGE_2 = "positive_change_2"
POSITIVE_CHANGE_3 = "positive_change_3"
CLASS = "class"

PREFIX_ONE = "A"
PREFIX_TWO = "B" 
PREFIX_THREE = "C" 

In [3]:
df = pd.read_csv(DIR_PATH + RESTRUCTURING_DATA_PATH)  # read in the data with profitability and restructuring frequency
df = df.sort_values(by=['Company_Key', 'Year'])       # sort the records by company number and year
df = df[COLUMNS_ORDER]                                # sort the colum by given order
df = df.drop_duplicates(subset=['Company_Key', 'Year'], keep='last')
print(df.shape)
df.head(15)

(429, 11)


,Company_Key,Year,compamy_name,profitability,profitability_next_1,profitability_next_2,profitability_next_3,change_next_1,change_next_2,change_next_3,is_restructuring
204,4904,2011,AEP Texas Central Transition Funding II LLC,0.0322,0.0255,0.0272,0.0221,-20.91,-15.54,-31.33,0
48,4904,2012,AMERICAN ELECTRIC POWER CO INC,0.0255,0.0272,0.0221,0.0420,6.79,-13.17,65.03,1
241,4904,2013,OHIO POWER CO,0.0272,0.0221,0.0420,0.0134,-18.69,54.54,-50.69,0
234,4904,2014,AEP Texas Central Transition Funding II LLC,0.0221,0.0420,0.0134,0.0561,90.07,-39.36,153.76,0
124,4904,2015,AMERICAN ELECTRIC POWER CO INC,0.0420,0.0134,0.0561,0.0291,-68.09,33.51,-30.70,1
37,4904,2016,AMERICAN ELECTRIC POWER CO INC,0.0134,0.0561,0.0291,0.0396,318.45,117.20,195.34,1
23,4904,2017,AMERICAN ELECTRIC POWER CO INC,0.0561,0.0291,0.0396,0.0770,-48.09,-29.42,37.29,1
77,4904,2018,AMERICAN ELECTRIC POWER CO INC,0.0291,0.0396,0.0770,0.0712,35.97,164.49,144.43,1
181,31978,2011,EL PASO ELECTRIC CO,0.0205,0.0127,0.0007,-0.0116,-38.06,-96.58,-156.66,0
378,31978,2012,EL PASO ELECTRIC CO /TX/,0.0127,0.0007,-0.0116,-0.0011,-94.47,-191.47,-108.78,0


In [4]:
# add positive change columns for three years, and set default values to zero
df[POSITIVE_CHANGE_1] = 0
df[POSITIVE_CHANGE_2] = 0
df[POSITIVE_CHANGE_3] = 0

# classify the values of positive changes as one, and negative changes as zero
df[POSITIVE_CHANGE_1] = df["change_next_1"].apply(lambda x: 1 if x > 0 else 0)
df[POSITIVE_CHANGE_2] = df["change_next_2"].apply(lambda x: 1 if x > 0 else 0)
df[POSITIVE_CHANGE_3] = df["change_next_3"].apply(lambda x: 1 if x > 0 else 0)

df.head(LIMIT)

,Company_Key,Year,compamy_name,profitability,profitability_next_1,profitability_next_2,profitability_next_3,change_next_1,change_next_2,change_next_3,is_restructuring,positive_change_1,positive_change_2,positive_change_3
204,4904,2011,AEP Texas Central Transition Funding II LLC,0.0322,0.0255,0.0272,0.0221,-20.91,-15.54,-31.33,0,0,0,0
48,4904,2012,AMERICAN ELECTRIC POWER CO INC,0.0255,0.0272,0.0221,0.0420,6.79,-13.17,65.03,1,1,0,1
241,4904,2013,OHIO POWER CO,0.0272,0.0221,0.0420,0.0134,-18.69,54.54,-50.69,0,0,1,0


In [5]:
# copy the original dataframe for three times for group classification in three different years
df_y1 = df.copy()
df_y2 = df.copy()
df_y3 = df.copy()

# put data and POSITIVE_CHANGE into a list of tuples for convenience in processing
dfs = [(df_y1, POSITIVE_CHANGE_1),
       (df_y2, POSITIVE_CHANGE_2),
       (df_y3, POSITIVE_CHANGE_3)]


# add positive change columns for three years, and set default values to zero
for df in dfs:
    df[0][CLASS] = 0

In [6]:
def get_class(df, year_gap, restructure_col, profit_col):

    if year_gap == 1:
        pre = PREFIX_ONE
    elif year_gap == 2:
       pre = PREFIX_TWO
    elif year_gap == 3:
       pre = PREFIX_THREE

    # define the filters for four categories
    filter_1 = (df[restructure_col] == 1) & (df[profit_col] == 0)
    filter_2 = (df[restructure_col] == 1) & (df[profit_col] == 1)
    filter_3 = (df[restructure_col] == 0) & (df[profit_col] == 0)
    filter_4 = (df[restructure_col] == 0) & (df[profit_col] == 1)


    df[CLASS][filter_1] = f"{pre}1"
    df[CLASS][filter_2] = f"{pre}2"
    df[CLASS][filter_3] = f"{pre}3"
    df[CLASS][filter_4] = f"{pre}4"

    return df

In [7]:
import warnings
warnings.filterwarnings("ignore")

# apply get_class function on three dataframes to classify rocords into four groups
for i in range(len(dfs)):
    df, change = dfs[i][0], dfs[i][1]
    df = df.pipe(get_class, i+1, IS_RESTRUCTURING, change)
    print(df[CLASS].value_counts())

A3    173
A4    133
A1     63
A2     60
Name: class, dtype: int64
B3    164
B4    142
B2     69
B1     54
Name: class, dtype: int64
C3    173
C4    133
C2     72
C1     51
Name: class, dtype: int64


## End of classification

## Start statistical test

In [8]:
from scipy import stats

In [9]:
class Test:

    def __init__(self, data, column):
        self.data = data
        
        arr = self.data[column].unique()
        arr.sort()

        self.column = column
        self.classes =arr

        self.independent_ttest_result = None
        self.paired_ttest_result = None


    def independent_ttest(self, column_to_compare):


        filter_12 = (self.data[self.column] == self.classes[0]) | (self.data[self.column] == self.classes[1])
        filter_34 = (self.data[self.column] == self.classes[2]) | (self.data[self.column] == self.classes[3])


        results = stats.ttest_ind(df_y1[filter_12][column_to_compare], df_y1[filter_34][column_to_compare])

        # print(results[0])
        # print(results[1])

        self.independent_ttest_result = results

    def paired_ttest(self, col1, col2):

        filter_12 = (self.data[self.column] == self.classes[0]) | (self.data[self.column] == self.classes[1])


        results = stats.ttest_rel(self.data[filter_12][col1], self.data[filter_12][col2])

        # print(results[0])
        # print(results[1])

        self.paired_ttest_result = results

    def get_test_result(self):
        # compare among four classes
        # between(independent t-test)
        # class1+2 vs. class 3+4(Does the impact of restructure on profitability change significant?)
        print("between(independent t-test)")
        print(self.independent_ttest_result)

        print("")

        # within(Paired t-test)
        # before the restructrue and after the restructure
        # class1+class2
        print("within(Paired t-test)")
        print(self.paired_ttest_result)

##1-year lag

In [10]:
# compare among class A1234
test1 = Test(df_y1, "class")
test1.independent_ttest("change_next_1")
test1.paired_ttest("profitability", "profitability_next_1")
test1.get_test_result()

between(independent t-test)
Ttest_indResult(statistic=-1.9078287862146015, pvalue=0.05708384239462489)

within(Paired t-test)
Ttest_relResult(statistic=-2.676806391650291, pvalue=0.008455212760692839)


## 2-year lag

In [11]:
# compare among class B1234
test2 = Test(df_y2, "class")
test2.independent_ttest("change_next_2")
test2.paired_ttest("profitability", "profitability_next_2")
test2.get_test_result()

between(independent t-test)
Ttest_indResult(statistic=-1.6189066411957396, pvalue=0.10620568653168308)

within(Paired t-test)
Ttest_relResult(statistic=-4.229568250135589, pvalue=4.55283740315995e-05)


## 3-year lag

In [12]:
# compare among class C1234
test3 = Test(df_y3, "class")
test3.independent_ttest("change_next_3")
test3.paired_ttest("profitability", "profitability_next_3")
test3.get_test_result()

between(independent t-test)
Ttest_indResult(statistic=0.2090219353498163, pvalue=0.8345307923206395)

within(Paired t-test)
Ttest_relResult(statistic=-3.5692528295867008, pvalue=0.0005128852008634215)
